# Measuring computational performance

There are two main dimensions of computational performance: execution time and memory usage. The one which you'll usually run into trouble with first, and which also happens to be easier to measure, is execution time. In this notebook we'll review some simple techniques for measuring execution time, and some very basic related Computer Science concepts.

## An illustrative example: sorting names

In the cell below, you'll find a list of names. We will use the alphabetical sorting of these names as an example of a task whose execution time we might want to measure.

In [1]:
list_of_names = ['Boyang','Aaron','Tianqi','Ivan','Hasan',
                 'Maryan','Neil','Yuto','Hao','Dimas',
                 'Deven','Carsten']

In the cell below, we define two functions:
  - `string_leq` returns `True` if the first string alphabetically precedes (or is the same as) the second string, and `False` otherwise
  - `string_list_sort` uses `string_leq` in a primitive sorting algorithm to alphabetize a list of strings

Notice that Python can already compare single characters using `>,=,<`, which we will take advantage of. Actually, it can also compare whole strings, but in order to make things more interesting, we do not take advantage of this functionality in this initial example.

In [12]:
## Test if the first string alphabetically precedes the second string
def string_leq(stringa,stringb):
    alen = len(stringa)
    blen = len(stringb)
    for i in range(alen):
        if (i + 1) > blen:
            return False
        elif stringa[i] < stringb[i]:
            return True
        elif stringa[i] > stringb[i]:
            return False
    return True

## Sort strings, using a primitive algorithm and the primitive string-comparison function from above
def string_list_sort(a_list):
    sorted = []
    for current_string in a_list:
        position = 0
        while position < len(sorted):
            if string_leq(current_string,sorted[position]):
                break
            position += 1
        sorted = sorted[0:position] + [current_string] + sorted[position:]
    return sorted

string_list_sort(list_of_names)

['Aaron',
 'Boyang',
 'Carsten',
 'Deven',
 'Dimas',
 'Hao',
 'Hasan',
 'Ivan',
 'Maryan',
 'Neil',
 'Tianqi',
 'Yuto']

One way to measure execution time which will work in almost any programming language is to use the clock's timer directly. In Python, to do this we should import the `time` library.

The difference between two timepoints will by default be given in seconds.

If we use this primitive method on an operation which completes very quickly, we might want to use a loop to repeat it many times, to minimize measurement error.

In [13]:
import time

start_time = time.time()
sorted_names = string_list_sort(list_of_names)
end_time = time.time()

print(f'Time for 1 run: {end_time-start_time:.16f} seconds.')

Nruns = 1000
start_time = time.time()
for j in range(Nruns):
    sorted_names = string_list_sort(list_of_names)
end_time = time.time()

print(f'Time for {Nruns:,d} runs: {end_time-start_time:.16f} seconds.')
print(f'Average time per run: {(end_time-start_time)/Nruns:.16f} seconds.')

Nruns = 10000
start_time = time.time()
for j in range(Nruns):
    sorted_names = string_list_sort(list_of_names)
end_time = time.time()

print(f'Time for {Nruns:,d} runs: {end_time-start_time:.16f} seconds.')
print(f'Average time per run: {(end_time-start_time)/Nruns:.16f} seconds.')



Time for 1 run: 0.0000000000000000 seconds.
Time for 1,000 runs: 0.0438826084136963 seconds.
Average time per run: 0.0000438826084137 seconds.
Time for 10,000 runs: 0.4178805351257324 seconds.
Average time per run: 0.0000417880535126 seconds.


When using Jupyter notebooks, an easier and more accurate way to measure execution time is with the `%%time` cell magic and the `%timeit` line magic commands.

`%%time` as the first list in a cell will measure the execution time of all the code in the cell, for one go.

`%timeit` followed by a line of code will repeat the execution of that line several times (many times, to increase accuracy, if execution is fast), and return the average execution time.

In [14]:
%%time

start_time = time.time()
for j in range(Nruns):
    sorted_names = string_list_sort(list_of_names)
end_time = time.time()

print(f'Time for {Nruns:,d} runs: {end_time-start_time:.16f} seconds.')
print(f'Average time per run: {(end_time-start_time)/Nruns:.16f} seconds.')


Time for 10,000 runs: 0.4236965179443359 seconds.
Average time per run: 0.0000423696517944 seconds.
CPU times: total: 422 ms
Wall time: 425 ms


In [15]:
%timeit sorted_names = string_list_sort(list_of_names)

37.8 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


As we can see, the answers here aren't too far off from what we get with the primitive method. And they also have random variation. But they are convenient and they also give us some additional information which is sometimes useful.

### Comparing with Python built-in functions

Sorting algorithms are the bread-and-butter of introductory computer science courses and have been very thoroughly studied and optimized. Python has built-in capability to sort lists of numbers and strings, which might be better-optimized than the quick, easy algorithm we coded above.

The built-in sorting function is `sorted()`. Let's see how it does:

In [16]:
print(sorted(list_of_names))

['Aaron', 'Boyang', 'Carsten', 'Deven', 'Dimas', 'Hao', 'Hasan', 'Ivan', 'Maryan', 'Neil', 'Tianqi', 'Yuto']


In [17]:
%timeit sorted_names = string_list_sort(list_of_names)

%timeit sorted_names = sorted(list_of_names)

43.4 µs ± 5.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
501 ns ± 5.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Our algorithm takes about 30 microseconds, whereas the built-in algorithm takes around 340 *nano*seconds. Three time 340 nanoseconds is approximately 1 microsecond, which means the built-in Python algorithm is about 90 times faster.

There are many reasons for this. To point out a couple of the ways in which our algorithm is sub-optimal:
  - it uses loops which in Python use a lot of overhead to manage
  - our algorithm has a nested loop which goes through the entire list, and then checks each element against each already-sorted element. There are algorithms which can achieve sorting using many fewer comparisons.

If you want to learn about sorting algorithms, take a computer science course! Or look them up on the internet--there's a lot of good free material.

### Performance variation on larger and larger inputs

A key aspect of an algorithm's execution time is how it will increase as the size of the input increases. Suppose we double the size of the list to be sorted. Will this:
  - double the execution time?
  - *less than* double the execution time?
  - *more than* double the execution time?

The answer can be different for every algorithm. Let's test it out with the two that we have:

In [18]:
%timeit sorted_names = string_list_sort(list_of_names*10)

%timeit sorted_names = sorted(list_of_names*10)

3.16 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
9.49 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In the case of our custom algorithm, multiplying the input size by 10 increased our execution time to 2.3 milliseconds, which is 2,300 *micro*seconds. In otherwords, it multiplied execution time by *a bit less than 100*--or $10^2$. This indicates that the *time complexity*, using "Big O" notation, is a bit less than $O(n^2)$.

It is not surprising that execution time scales by roughly the square if input size, because of the nested loop structure of our algorithm--it goes through the whole list, and then checks each element against each element in the sorted list.

In constrast, the built-in algorithm has a time complexity much closer to $O(n)$: after increasing the input size by 10, the execution time was multiplied by 10 or 20.

If we had to sort a very long list, our custom algorithm would become infeasible at a point when the performance of the built-in algorithm would still be very strong.

Let's try multiplying the input size by 500:



In [19]:
%timeit sorted_names = string_list_sort(list_of_names*500)

%timeit sorted_names = sorted(list_of_names*500)

7.09 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
639 µs ± 7.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Execution time for our custom algorithm is now *several* whole seconds. The built-in algorithm still sorts this list of 6,000 names in just about 1 half of 1 thousandth of a second: faster than we're even able to notice.

### Performance variation with *pattern* of input

Sometimes it is not just the size of the input but patterns exhibited by the input that can increase execution time.

Consider string comparisons, for example. The number of characters that must be considered to pick a "winner" is variable. If the first character is the same, the second is a tie-breaker, and if the second is the same, the third, etc.

Our string-comparison function that we've written takes this very simple, linear approach. But so far it hasn't had any need to go beyond the first characters: all the names in the list we've been sorting start with different letters.

What if we had names which were much more similar?

Below we have two more lists of 12 names. For the first list, all elements share the first 2 letters. For the second, they share the first 4.

In [20]:


list_of_ja_names = ['Jay','Jade','James','Jana','Jaan',
                        'Jack','Jackson','Jacob','Jane','Jason',
                        'Jasper','Jayden']


list_of_mich_names = ['Michaela','Michela','Michelle','Michaella','Michaiah',
                      'Michaelia','Michalina','Miche','Michelina','Michael',
                      'Michigan','Michelin'
]

### Quick exercise for measuring performance:
 - Test how the performance of the custom algorithm changes for the different lists of names. Does the pattern make sense? How would you explain it?
 - Test how the performance of the Python built-in algorithm changes for the different lists of names. Does the pattern make sense? How would you explain it?
